In [1]:
import pandas as pd
from bokeh.models import ColumnDataSource, LabelSet, Title, Axis
from bokeh.plotting import figure, show
from bokeh.sampledata.sprint import sprint
from bokeh.plotting import figure, show
from bokeh.layouts import gridplot
import bokeh.io
from bokeh.io import curdoc
from bokeh.themes import Theme
from bokeh.models import CustomJS, Dropdown
from bokeh.models import PrintfTickFormatter
from bokeh.models import NumeralTickFormatter



bokeh.io.reset_output()
bokeh.io.output_notebook()
pd.set_option('display.float_format', lambda x: '%.5f' % x)

Loading BokehJS ...

In [2]:
df = pd.read_csv("../data/all_2021_nfl_contracts.csv")

In [3]:
df["contract_length"] = pd.to_numeric(df.contract_length.replace("-", None))

df["contract_expires"] = df.contract_expires.replace("-", None)
df["contract_expires"] = pd.to_numeric(df.contract_expires)

df["contract_start"] = df.contract_expires - df.contract_length
df['_contract_amount'] = df.contract_amount.map('${:,.0f}'.format)
df['_guaranteed_amount'] = pd.to_numeric(df.guaranteed_amount.replace("-",None).str.replace("$", "").str.replace(",",""))
df["contract_remaining_years"] = df.contract_expires - 2022

/tmp/ipykernel_2142715/4021721831.py:8: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df['_guaranteed_amount'] = pd.to_numeric(df.guaranteed_amount.replace("-",None).str.replace("$", "").str.replace(",",""))


# Bokeh

In [5]:
tooltips = [
    ("Team", "@tm"),
]

p = figure(width=1000, height=1000, title="Guarnteed Contract Amount vs Average Remaining Contract Years", tooltips=tooltips,)


colormap = {'chicago-bears': '#c83803'}
colors = [colormap.get(x, "white") for x in df.tm.unique()]


source = ColumnDataSource(df[(df.contract_amount > 1000000)].groupby("tm", as_index=False).mean()[["tm","contract_remaining_years", "_guaranteed_amount"]])
p.circle(source = source, x="contract_remaining_years", y="_guaranteed_amount", size=15, color="white")

p.title.text_font_size = "25px"
p.title.text_color ="#c83803"
p.title.align = 'center'
p.title.vertical_align = 'bottom'
p.min_border_top = 75
p.min_border_bottom = 75
p.yaxis.major_label_text_color = "#c83803"
p.xaxis.major_label_text_color = "#c83803"
p.background_fill_color = "#0B162A"
p.border_fill_color = "#0B162A"
p.ygrid.grid_line_alpha = 0.2
p.ygrid.grid_line_dash = [6, 4]
p.axis.axis_line_color = None
p.yaxis.major_label_text_font_size = "13pt"

p.xaxis.major_label_text_font_size = "13pt"
p.xaxis.axis_label_text_font_size = "15pt"
p.xaxis.axis_label_text_color = "#c83803"


p.yaxis.major_label_text_font_size = "13pt"
p.yaxis.axis_label_text_font_size = "15pt"
p.yaxis.axis_label_text_color = "#c83803"


p.xaxis.axis_label="Avg Remaining Contract Years"
p.yaxis.axis_label="Avg Guaranteed Contract Amount"

#p.yaxis[0].formatter = PrintfTickFormatter(format="%{:,}")
p.yaxis[0].formatter = NumeralTickFormatter(format="‘0.0a’")


#p.add_layout(labels)
 
show(p)

# Altair

In [4]:
df["remaining_guaranteed_contract_amount"] = df._guaranteed_amount / df.contract_remaining_years

In [5]:
df = df[(df.contract_amount > 1000000)].groupby("tm", as_index=False).agg({"contract_remaining_years":"mean", "remaining_guaranteed_contract_amount":"sum"})[["tm","contract_remaining_years", "remaining_guaranteed_contract_amount"]]
df["Is Bears"] = df.tm == "chicago-bears"

In [6]:
import altair as alt
from vega_datasets import data
alt.Chart(df, 
          width=1000, 
          height=1000, 
          title="Chicago Bears Remaining Guaranteed Contract Amount vs Average Remaining Contract Years").mark_circle(
    size=200, 
    opacity=1).encode(text="tm",
    x=alt.X('contract_remaining_years',title='Avg Contract Remaining Years',
        scale=alt.Scale(zero=False)
    ),
    y=alt.Y('remaining_guaranteed_contract_amount',title="Total Remaining Guaranteed Contract Amount",
        scale=alt.Scale(zero=False)
    ),
    color=alt.Color("Is Bears", 
                    scale=alt.Scale(domain=[True, False], range=["#c83803", "white"])
                   ),
    tooltip=[alt.Tooltip("tm", title="Team Name"), 
             alt.Tooltip("remaining_guaranteed_contract_amount", format=",.0f", title="Guaranteed Amount"), 
             alt.Tooltip("contract_remaining_years", format=",.2f", title=" Contract Remaining Years")
            ]
).configure(background='#0B162A').configure_axis(gridColor="grey", gridDash=[6,4]).configure_axisLeft(
  labelColor='#c83803',
  titleColor='#c83803',
labelFontSize=15, titleFontSize=17
).configure_axisBottom(
  labelColor='#c83803',
  titleColor='#c83803',
    labelFontSize=15, titleFontSize=17
).configure_title(color="#c83803", fontSize=25).configure_legend(labelColor="#c83803", titleColor="#c83803",                                                             
    strokeColor='gray',
    fillColor='#0B162A',
    padding=10,
    cornerRadius=10,
    orient='top-right', labelFontSize=15, titleFontSize=15)

alt.Chart(...)